# Instalación de paquetes

In [1]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate
!apt install git-lfs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.8 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 4.7 MB/s eta 

# Login

2.1 Storage HF token

In [ ]:
!git config --global user.email "jose.bucheli@correounivalle.edu.co"
!git config --global user.name "TheCryss"

In [2]:
from huggingface_hub import notebook_login

notebook_login()

# 2.2 Load of the dataset

In [31]:
from datasets import load_dataset

raw_datasets = load_dataset("conll2002", 'es')

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1491: FutureWarning: The repository for conll2002 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conll2002
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/8324 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1916 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1518 [00:00<?, ? examples/s]

Viewing token and ner entities

In [32]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'ner_tags'],
        num_rows: 8324
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'ner_tags'],
        num_rows: 1916
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'ner_tags'],
        num_rows: 1518
    })
})

In [33]:
raw_datasets["train"][0]["tokens"]

['Melbourne', '(', 'Australia', ')', ',', '25', 'may', '(', 'EFE', ')', '.']

In [34]:
raw_datasets["train"][0]["ner_tags"]

[5, 0, 5, 0, 0, 0, 0, 0, 3, 0, 0]

In [35]:
raw_datasets["train"].features["ner_tags"]

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

In [36]:
ner_feature = raw_datasets["train"].features["ner_tags"]
ner_feature

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

In [37]:
label_names = ner_feature.feature.names
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

# 2.3.1  Define tokenize (BETO)

In [38]:
from transformers import AutoTokenizer

model_checkpoint = "NazaGara/NER-fine-tuned-BETO"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [39]:
tokenizer.is_fast

True

In [40]:
inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)
inputs.tokens()

['[CLS]',
 'Mel',
 '##bourne',
 '(',
 'Australia',
 ')',
 ',',
 '25',
 'may',
 '(',
 'EF',
 '##E',
 ')',
 '.',
 '[SEP]']

In [41]:
inputs.word_ids()

[None, 0, 0, 1, 2, 3, 4, 5, 6, 7, 8, 8, 9, 10, None]

In [42]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [43]:
labels = raw_datasets["train"][0]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(word_ids)

print(align_labels_with_tokens(labels, word_ids))

[5, 0, 5, 0, 0, 0, 0, 0, 3, 0, 0]
[None, 0, 0, 1, 2, 3, 4, 5, 6, 7, 8, 8, 9, 10, None]
[-100, 5, 6, 0, 5, 0, 0, 0, 0, 0, 3, 4, 0, 0, -100]


In [44]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [45]:
tokenize_and_align_labels(raw_datasets['train'][:5])

{'input_ids': [[4, 7762, 24537, 1147, 7115, 1135, 1017, 2329, 1582, 1147, 18122, 30956, 1135, 1009, 5], [4, 1149, 5], [4, 1162, 20731, 2088, 1072, 2115, 1017, 5650, 12042, 1005, 14492, 1017, 22097, 2456, 1030, 3531, 1008, 3087, 2160, 1110, 4409, 1091, 2116, 6724, 25946, 2999, 1013, 1108, 5075, 1008, 11665, 1038, 22822, 2158, 1095, 1040, 4570, 1008, 1065, 3770, 7797, 1008, 1030, 2826, 1009, 5], [4, 1198, 6894, 1072, 20731, 2088, 1512, 1675, 1888, 1008, 1038, 1049, 7874, 1072, 3716, 12938, 1072, 1814, 1008, 11527, 1147, 7115, 1135, 1062, 19811, 21483, 1013, 30043, 1049, 10207, 6362, 1042, 16666, 1040, 2714, 2379, 1040, 12945, 1008, 1030, 5105, 1008, 1038, 1089, 1858, 1038, 1114, 4739, 1539, 8540, 1987, 10467, 1924, 1269, 1040, 10556, 1013, 2718, 1008, 1030, 5075, 4095, 1258, 30969, 1710, 1009, 5], [4, 1949, 5075, 1004, 3953, 4418, 1049, 1944, 1008, 5731, 1017, 1577, 1036, 1040, 1038, 1131, 1790, 6054, 1093, 1036, 1216, 1008, 1049, 8172, 1008, 6665, 1017, 1042, 18080, 1924, 1269, 6452, 10

In [46]:
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

Map:   0%|          | 0/8324 [00:00<?, ? examples/s]

Map:   0%|          | 0/1916 [00:00<?, ? examples/s]

Map:   0%|          | 0/1518 [00:00<?, ? examples/s]

# 2.3.2 padding

In [47]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [48]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
batch["labels"]

tensor([[-100,    5,    6,    0,    5,    0,    0,    0,    0,    0,    3,    4,
            0,    0, -100],
        [-100,    0, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100]])

In [49]:
!pip install seqeval

In [50]:
import evaluate

metric = evaluate.load("seqeval")

In [51]:
labels = raw_datasets["train"][0]["ner_tags"]
labels = [label_names[i] for i in labels]
labels

['B-LOC', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O']

In [52]:
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

# 2.4 Finetuning

In [53]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

In [54]:
model.config.num_labels

9

# 2.5 Train strategy

In [60]:
from transformers import TrainingArguments

repositoryName = "NER-finetuned-BETO"
args = TrainingArguments(
    repositoryName,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=10,
    weight_decay=0.01,
    push_to_hub=True,
    #batch_eval_metrics=True,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=8
)

# 2.6 Implementación del Trainer

In [61]:
import evaluate
import numpy as np

def compute_metrics(eval_preds):
  logits, labels = eval_preds
  predictions = np.argmax(logits, axis=-1)

  true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
  true_predictions = [
      [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
      for prediction, label in zip(predictions, labels)
  ]

  all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
  return {
      "precision": all_metrics["overall_precision"],
      "recall": all_metrics["overall_recall"],
      "f1": all_metrics["overall_f1"],
      "accuracy": all_metrics["overall_accuracy"]
  }

# 2.7 Entrenamiento

In [62]:
from transformers import Trainer

trainer = Trainer(

    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [63]:
trainer.train()


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.010400,0.191514,0.835911,0.856847,0.846250,0.970121
2,0.010100,0.218667,0.822628,0.838695,0.830584,0.967626
3,0.006600,0.208491,0.855096,0.863741,0.859396,0.969920
4,0.006900,0.213860,0.834243,0.843061,0.838629,0.969843
5,0.007000,0.211013,0.847980,0.853631,0.850796,0.970819
6,0.006000,0.221389,0.837789,0.849724,0.843714,0.970276
7,0.004200,0.228395,0.843708,0.859605,0.851582,0.970431
8,0.003400,0.234441,0.841725,0.856618,0.849106,0.970152
9,0.002600,0.238486,0.840045,0.857996,0.848926,0.969781
10,0.002300,0.241187,0.846015,0.860983,0.853434,0.970400


TrainOutput(global_step=5210, training_loss=0.005793518474372015, metrics={'train_runtime': 1626.662, 'train_samples_per_second': 51.172, 'train_steps_per_second': 3.203, 'total_flos': 3617590609959768.0, 'train_loss': 0.005793518474372015, 'epoch': 10.0})

# Publicación de modelo HF

In [66]:
from huggingface_hub import ModelCard, whoami

metrics = trainer.state.log_history

training_details = {
    "epochs": args.num_train_epochs,
    "learning_rate": args.learning_rate,
    "weight_decay": args.weight_decay,
    "train_batch_size": args.train_batch_size,
    "eval_batch_size": args.eval_batch_size
}

metrics_data = "| Epoch | Training Loss | Validation Loss | Precision | Recall | F1 Score | Accuracy |\n"
metrics_data += "|:----:|:-------------:|:---------------:|:---------:|:------:|:--------:|:--------:|\n"

for entry in metrics:
    if 'loss' in entry:
        epoch = round(entry['epoch'])
        train_loss = entry['loss']
        metrics_data += f"| {epoch} | {train_loss:.4f}"

    elif 'eval_loss' in entry:
      val_loss = entry['eval_loss']
      precision = entry.get('eval_precision', 'N/A')
      recall = entry.get('eval_recall', 'N/A')
      f1 = entry.get('eval_f1', 'N/A')
      accuracy = entry.get('eval_accuracy', 'N/A')
      metrics_data += f"| {val_loss:.4f} | {precision:.4f} | {recall:.4f} | {f1:.4f} | {accuracy:.4f} |\n"


print(metrics_data)

card_content = f"""
---
license: apache-2.0
datasets:
- eriktks/conll2002
language:
- es
metrics:
- precision
- recall
- f1
- accuracy
pipeline_tag: token-classification
---

# Model Name: {repositoryName}

This is a BERT model fine-tuned for Named Entity Recognition (NER).

# Model Description

This is a fine-tuned BERT model for Named Entity Recognition (NER) task using CONLL2002 dataset.

In the first part, the dataset must be pre-processed in order to give it to the model. This is done using the 🤗 Transformers and BERT tokenizers. Once this is done, finetuning is applied from *[BETO](https://huggingface.co/NazaGara/NER-fine-tuned-BETO)* and using the 🤗 *AutoModelForTokenClassification*.

Finally, the model is trained obtaining the neccesary metrics for evaluating its performance (Precision, Recall, F1 and Accuracy)

Summary of executed tests can be found in: https://docs.google.com/spreadsheets/d/1lI7skNIvRurwq3LA5ps7JFK5TxToEx4s7Kaah3ezyQc/edit?usp=sharing

Model can be found in: https://huggingface.co/Seb00927/NER-finetuned-BETO

Github repository: https://github.com/paulrojasg/nlp_4th_workshop

# Training

## Training Details

- Epochs: {training_details['epochs']}
- Learning Rate: {training_details['learning_rate']}
- Weight Decay: {training_details['weight_decay']}
- Batch Size (Train): {training_details['train_batch_size']}
- Batch Size (Eval): {training_details['eval_batch_size']}

## Training Metrics

{metrics_data}

# Authors
Made by:
- Paul Rodrigo Rojas Guerrero
- Jose Luis Hincapie Bucheli
- Sebastián Idrobo Avirama

With help from:
- [Raúl Ernesto Gutiérrez](https://huggingface.co/raulgdp)
"""

model_card = ModelCard(card_content)

user = whoami()['name']

model_card.push_to_hub(repo_id=f'{user}/{repositoryName}')

| Epoch | Training Loss | Validation Loss | Precision | Recall | F1 Score | Accuracy |
|:----:|:-------------:|:---------------:|:---------:|:------:|:--------:|:--------:|
| 1 | 0.0104| 0.1915 | 0.8359 | 0.8568 | 0.8462 | 0.9701 |
| 2 | 0.0101| 0.2187 | 0.8226 | 0.8387 | 0.8306 | 0.9676 |
| 3 | 0.0066| 0.2085 | 0.8551 | 0.8637 | 0.8594 | 0.9699 |
| 4 | 0.0069| 0.2139 | 0.8342 | 0.8431 | 0.8386 | 0.9698 |
| 5 | 0.0070| 0.2110 | 0.8480 | 0.8536 | 0.8508 | 0.9708 |
| 6 | 0.0060| 0.2214 | 0.8378 | 0.8497 | 0.8437 | 0.9703 |
| 7 | 0.0042| 0.2284 | 0.8437 | 0.8596 | 0.8516 | 0.9704 |
| 8 | 0.0034| 0.2344 | 0.8417 | 0.8566 | 0.8491 | 0.9702 |
| 9 | 0.0026| 0.2385 | 0.8400 | 0.8580 | 0.8489 | 0.9698 |
| 10 | 0.0023| 0.2412 | 0.8460 | 0.8610 | 0.8534 | 0.9704 |



CommitInfo(commit_url='https://huggingface.co/Seb00927/NER-finetuned-BETO/commit/01e9d7174c3104123a320c5ac5102fc291b08748', commit_message='Upload README.md with huggingface_hub', commit_description='', oid='01e9d7174c3104123a320c5ac5102fc291b08748', pr_url=None, pr_revision=None, pr_num=None)